## RECLASSIFICAR + RASTERIZAR + tqdm + chunck + salvamentos parciais

In [1]:
import geopandas as gpd
import rasterio
from rasterio import features
from rasterio.transform import from_bounds
import rasterio.windows
import fiona  # Usado para ler os bounds de forma eficiente
import os
from tqdm import tqdm

# --- 1. CONFIGURAÇÃO ---
# Caminho para o seu arquivo GeoPackage completo do Brasil
input_gpkg_path = r'BRA_AD_Solos_4326.gpkg'

# Caminho onde o raster de saída (GeoTIFF) será salvo
output_tif_path = r'BASES\AD_Solos_30m.tif'

# Resolução do raster em graus (aproximadamente 30 metros)
resolution = 0.00027

# Valor para pixels que não são cobertos por nenhum polígono (background)
# --- MUDANÇA 1: Alterado de 0 para 255 para evitar conflito com AD0 ---
nodata_value = 255 

# Mapeamento de valores
# --- MUDANÇA 2: Adicionado 'AD0': 0 ao mapa ---
value_map = {
    'AD0': 7, 'AD1': 1, 'AD2': 2, 'AD3': 3,
    'AD4': 4, 'AD5': 5, 'AD6': 6
}

# --- NOVO: Tamanho do Chunk (Tile) ---
# Vamos processar o raster em blocos de 512x512 pixels
tile_size = 512

# --- 2. LEITURA EFICIENTE DOS METADADOS (Bounds e CRS) ---

print(f"Lendo metadados (bounds e CRS) de: {input_gpkg_path}...")
try:
    with fiona.open(input_gpkg_path, 'r') as src:
        minx, miny, maxx, maxy = src.bounds
        # Verifica o CRS
        input_crs = src.crs
        if input_crs['init'] != 'epsg:4326':
            raise Exception(f"Erro: O CRS do arquivo é {input_crs['init']}, mas precisa ser 'epsg:4326' (WGS 84).")
except Exception as e:
    print(f"Erro ao ler o GeoPackage. Verifique o caminho e o CRS.")
    print(e)
    exit()

print("Metadados lidos com sucesso.")

# --- 3. DEFINIÇÃO DOS PARÂMETROS DO RASTER DE SAÍDA ---

# Calculando as dimensões totais do raster
width = int((maxx - minx) / resolution)
height = int((maxy - miny) / resolution)
transform = from_bounds(minx, miny, maxx, maxy, width, height)

print(f"Dimensões totais do raster de saída: {width} x {height} pixels.")

# Definindo o perfil (metadados) do arquivo GeoTIFF de saída
# 'nodata' agora usará o nodata_value (255) definido acima
profile = {
    'driver': 'GTiff',
    'height': height,
    'width': width,
    'count': 1,
    'dtype': rasterio.uint8,
    'crs': 'EPSG:4326',
    'transform': transform,
    'nodata': nodata_value, # <-- Isso agora será 255
    'compress': 'lzw',
    'tiled': True,
    'blockxsize': tile_size,
    'blockysize': tile_size
}

# --- 4. PROCESSAMENTO E RASTERIZAÇÃO EM "CHUNKS" (TILES) ---

os.makedirs(os.path.dirname(output_tif_path), exist_ok=True)

try:
    with rasterio.open(output_tif_path, 'w', **profile) as dst:
        # Pega a lista de janelas (windows) para iterar
        windows = [window for ij, window in dst.block_windows(1)]

        print(f"Iniciando rasterização em {len(windows)} tiles...")

        # Iteramos sobre cada "chunk" (tile/window)
        for window in tqdm(windows, desc="Rasterizando Tiles"):

            # 1. Calcula a transformação e os bounds geográficos deste tile
            tile_transform = rasterio.windows.transform(window, dst.transform)
            tile_bounds = rasterio.windows.bounds(window, dst.transform)

            # 2. Lê APENAS o vetor que cruza este tile (a mágica do "chunk")
            try:
                gdf_chunk = gpd.read_file(
                    input_gpkg_path,
                    bbox=tile_bounds
                )
            except Exception as e:
                print(f"Aviso: Erro ao ler chunk (pode ser um tile vazio): {e}")
                continue # Pula para o próximo tile

            # Se não achou nada, pula
            if gdf_chunk.empty:
                continue

            # 3. Filtra e Mapeia os dados (somente para este chunk)
            # --- MUDANÇA 3: Corrigido para copiar o chunk inteiro antes de mapear ---
            gdf_filtered = gdf_chunk.copy()

            if gdf_filtered.empty:
                continue

            gdf_filtered['raster_val'] = gdf_filtered['ClasseAD'].map(value_map)
            
            # Este dropna agora só vai remover valores que NÃO estão no mapa
            # (ex: 'AD7', 'AD8', etc., se existirem)
            gdf_filtered.dropna(subset=['raster_val'], inplace=True)
            gdf_filtered['raster_val'] = gdf_filtered['raster_val'].astype(int)

            if gdf_filtered.empty:
                continue

            # 4. Prepara as geometrias para rasterizar
            shapes = ((geom, val) for geom, val in zip(gdf_filtered.geometry, gdf_filtered.raster_val))

            # 5. Rasteriza apenas este tile
            tile_array = features.rasterize(
                shapes=shapes,
                out_shape=(window.height, window.width), # Tamanho do tile
                transform=tile_transform, # Transformação do tile
                fill=nodata_value, # <-- Importante: usa o 255 para o fundo
                dtype=rasterio.uint8
            )

            # 6. Escreve este tile rasterizado DIRETAMENTE no arquivo .tif
            dst.write(tile_array, 1, window=window)

    print("\nProcesso finalizado com sucesso!")
    print(f"Arquivo TIF em chunks salvo em: {output_tif_path}")

except Exception as e:
    print(f"\nOcorreu um erro durante a rasterização:")
    print(e)
    # Se der erro, apaga o arquivo incompleto
    if os.path.exists(output_tif_path):
        os.remove(output_tif_path)

Lendo metadados (bounds e CRS) de: BRA_AD_Solos_4326.gpkg...
Metadados lidos com sucesso.
Dimensões totais do raster de saída: 167168 x 144521 pixels.
Iniciando rasterização em 92541 tiles...


Rasterizando Tiles: 100%|██████████| 92541/92541 [1:00:42<00:00, 25.41it/s] 


Processo finalizado com sucesso!
Arquivo TIF em chunks salvo em: BASES\AD_Solos_30m.tif
